In [ ]:
import pandas as pd
import glob

import praw
import urllib.request
import json

from datetime import datetime
from datetime import date
import time


reddit = praw.Reddit(
    client_id="kRAJROlrIHaWMw",
    client_secret="f1EXDzFpTw1AryR_3urQ9O6I3AQ",
    user_agent="AITA_scrape",
)

In [ ]:
def unix_to_date(unix):
    str_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(unix))
    date_time = datetime.strptime(str_time, "%Y-%m-%d %H:%M:%S")
    return date_time

def praw_recent_posts():
    posts = []
    for x in ["week", "month", "year"]:
        count = 0
        for submission in reddit.subreddit('amitheasshole').top(x, limit=1000):
            posts.append([
                submission.title,
                submission.id,
                unix_to_date(submission.created_utc),
                submission.link_flair_text,
                submission.selftext,
                submission.num_comments,
                submission.score,
                submission.upvote_ratio,
            ])
            count += 1
            if count % 300 == 0:
                print(x,": Completed",count)
    post_df = pd.DataFrame(posts, columns=["title", "id", "date", "verdict", "body", "num_comments", "score", "upvote_ratio"])
    post_df.drop_duplicates(subset="id", inplace=True)
    post_df.to_csv("data/raw/to_combine/praw_recent_%s.csv" % date.today().strftime("%Y%m%d"))
    return post_df

praw_rec_df = praw_recent_posts()
len(praw_rec_df)

In [ ]:
def combine_new_praw_data():
    files_to_combine = glob.glob("data/raw/to_combine/*.csv")
    temp_list = []

    for filename in files_to_combine:
        df = pd.read_csv(filename, index_col=0, header=0)
        temp_list.append(df)
    full_df = pd.concat(temp_list, axis=0, ignore_index=True)

    print(len(full_df))
    full_df.drop_duplicates(subset="id", inplace=True)
    full_df.reset_index(drop=True, inplace=True)
    print(len(full_df))

    full_df.to_csv("data/interim/aita_scrapped_full.csv")
    return full_df

full_df = combine_new_praw_data()
print(full_df.shape)
full_df